In [1]:
%matplotlib qt
import numpy as np
import pandas as pd
from pandas import DataFrame
from scipy import stats
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import gc
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings('ignore')

path = '../data/Debt issuing company 2018 report/'

In [2]:
def drop_out(frame,col,model='Confidence interval',t_alpha=0.95,alpha=2,IQR_rate=1.5,quantile=0.95):
    '''modle: 'gauss','box','quantile' '''
    
    if model == 'Confidence interval':
        u_ = frame[col].mean()
        v_ = frame[col].std()
        interval_ = stats.t.interval(t_alpha,frame[col].count()-1,u_,v_)
        cond_ = (frame[col]<interval_[1])&(frame[col]>interval_[0])
    
    elif model == 'gauss':
        u_ = frame[col].mean()
        v_ = frame[col].std()
        cond_ = np.abs((frame[col]-u_)/v_) < alpha
    
    elif model == 'box':
        q1 = frame[col].quantile(0.25)
        q3 = frame[col].quantile(0.75)
        IQR = (q3-q1)*IQR_rate
        q1 -= IQR ; q3 += IQR
        cond_ = (frame[col]<q3)&(frame[col]>q1)
    
    elif model == 'quantile':
        top_ = frame[col].quantile(quantile)
        bottom_ = frame[col].quantile(1-quantile)
        cond_ = (frame[col]<top_)&(frame[col]>bottom_)
    
    else:
        print('please try again')
        return frame
    
    index_ = np.where(frame[col]!=frame[col],True,
                                                  np.where(cond_,True,False))
    frame = frame.loc[index_,:]
    return frame

In [3]:
def fenbutu(df):
    for i in range(df.shape[1]-2):
        ax = plt.subplot(5,5,i+1)
        ax.scatter(range(df.shape[0]),df.iloc[:,i+2].sort_values(),s=3)
        plt.title(df.columns[i+2])
    plt.suptitle('散点趋势图')

In [4]:
start_year = 2014
end_year = 2017
data_a = DataFrame()
for i in range(start_year,end_year+1):
    if i % 3 ==0:
        print('is concating {} {}/{}'.format(i,i-start_year+1,end_year+1-start_year))
    try:
        path_a = path+f'y/{i}y.xlsx'
        data_a_ = pd.read_excel(path_a)[:-2]
        data_a_.drop(['是否经过审计','审计意见']+[i for i in data_a_.columns if i.find('E')!=-1],axis=1,inplace=True)
    #     data_a_.loc[:,['主营业务收入(亿元)','主营业务利润(亿元)','净利润(亿元)']] = \
    #     data_a_.loc[:,['主营业务收入(亿元)','主营业务利润(亿元)','净利润(亿元)']].apply(lambda x:x/data_a_['报告期'].dt.month)
        data_a = pd.concat([data_a,data_a_])
    except:
        print(f'no {i}')
        pass
del data_a_
gc.collect()
print('finish concat data_y')

print(np.array(list(data_a.isnull().sum(0))))

is concating 2016 3/4
finish concat data_y
[   0    0  144 1739  183  334 2191 2191  478  633  545  268  540  441
  610  762 2195  598 1317 2034 2682  556 2194 2198 2166 1742 5213]


In [5]:
data_a.dropna(how='any',inplace=True)

In [6]:
ci = pd.read_excel(path+'comp_feature/产业类发债企业行业分类0827.xlsx',sheet_name='城投类企业')
ci = ci[['名称', '最新评级', '企业性质', '是否上市','一级分类', '二级分类']]
ci['trade'] = 1

In [7]:
tp = pd.read_excel(path+'comp_feature/产业类发债企业行业分类0827.xlsx',sheet_name='交通运输')
tp = tp[['名称', '最新评级', '企业性质', '是否上市','一级分类', '二级分类']]
tp['trade'] = 2

In [8]:
indestry = pd.read_excel(path+'comp_feature/产业类发债企业行业分类0827.xlsx',sheet_name='产业类企业')
indestry = indestry[['名称', '最新评级', '企业性质', '是否上市','一级分类', '二级分类']]
indestry['trade'] = 3

In [9]:
fi = pd.read_excel(path+'comp_feature/产业类发债企业行业分类0827.xlsx',sheet_name='金融类企业')
fi = tp[['名称', '最新评级', '企业性质', '是否上市','一级分类', '二级分类']]
fi['trade'] = 4

In [10]:
all_com = pd.concat([ci,tp,indestry,fi])[ci.columns]

In [11]:
ci_ = data_a.merge(ci[['名称',]],on='名称',)
tp_ = data_a.merge(tp[['名称',]],on='名称',)
indestry_ = data_a.merge(indestry[['名称',]],on='名称',)
fi_ = data_a.merge(fi[['名称',]],on='名称',)

In [12]:
for k in range(2):
    col_ = np.random.choice(ci_.columns[2:],len(ci_.columns[2:]),replace=False)
    for j in col_:
        ci_ = drop_out(ci_,j,model='gauss',alpha=3)
        tp_ = drop_out(tp_,j,model='gauss',alpha=3)
        indestry_ = drop_out(indestry_,j,model='gauss',alpha=3)

In [13]:
for col_ in ci_.columns[2:]:
    ci_.loc[:,col_] = ci_.loc[:,col_] - ci_.loc[:,col_].mean()
    tp_.loc[:,col_] = tp_.loc[:,col_] - tp_.loc[:,col_].mean()
    indestry_.loc[:,col_] = indestry_.loc[:,col_] - indestry_.loc[:,col_].mean()
    fi_.loc[:,col_] = fi_.loc[:,col_] - fi_.loc[:,col_].mean()

In [14]:
ci_.describe()

,总资产(亿元),货币资产(亿元),净资产(亿元),总债务(亿元),带息债务(亿元),净债务(亿元),经营活动现金流(亿元),投资活动现金流(亿元),筹资活动现金流(亿元),主营业务收入(亿元),...,净资产回报率(%),流动比率,速动比率,存货周转率,资产负债率,短期债务/总债务,带息债务/总投入资本,货币资金/短期债务,货币资金/总债务,获息倍数
count,2.158000e+03,2.158000e+03,2.158000e+03,2.158000e+03,2.158000e+03,2.158000e+03,2.158000e+03,2.158000e+03,2.158000e+03,2.158000e+03,...,2.158000e+03,2.158000e+03,2.158000e+03,2.158000e+03,2.158000e+03,2.158000e+03,2.158000e+03,2.158000e+03,2.158000e+03,2.158000e+03
mean,5.687634e-13,-2.854271e-15,-2.371659e-14,1.065189e-13,-4.688002e-14,1.468005e-14,-1.419110e-15,-5.729121e-15,-6.760528e-15,1.179861e-14,...,-1.310866e-16,3.015609e-15,1.023895e-15,5.209508e-16,7.273350e-14,-8.164327e-14,-6.114520e-14,9.707506e-16,1.978517e-16,1.196695e-14
std,7.754308e+01,1.057353e+01,3.831201e+01,4.833849e+01,3.373368e+01,2.742204e+01,8.801808e+00,5.379827e+00,1.168438e+01,5.569113e+00,...,1.350341e+00,2.674025e+00,1.002471e+00,2.061806e+00,1.328424e+01,1.670910e+01,1.459014e+01,3.692440e-01,1.035915e-01,9.243156e+01
min,-1.398709e+02,-1.244473e+01,-6.854198e+01,-7.120154e+01,-4.526775e+01,-4.184513e+01,-3.136531e+01,-2.120696e+01,-3.598823e+01,-8.553200e+00,...,-4.133220e+00,-4.120240e+00,-1.683356e+00,-3.704134e-01,-3.830098e+01,-4.021521e+01,-3.511235e+01,-4.883326e-01,-1.760734e-01,-3.077522e+01
25%,-6.045668e+01,-7.637564e+00,-2.677955e+01,-3.799999e+01,-2.581871e+01,-2.151672e+01,-4.263384e+00,-1.927187e+00,-7.872297e+00,-3.782148e+00,...,-9.593447e-01,-1.997365e+00,-7.567810e-01,-3.055384e-01,-9.065604e+00,-1.270696e+01,-1.065247e+01,-2.712178e-01,-7.975440e-02,-2.228495e+01
50%,-1.277807e+01,-3.301008e+00,-8.220352e+00,-9.773641e+00,-8.750459e+00,-7.380951e+00,1.495605e+00,1.808913e+00,-1.941821e+00,-1.288612e+00,...,-1.555197e-01,-7.105896e-01,-2.050060e-01,-2.568134e-01,5.968713e-01,-1.650607e+00,-6.160953e-01,-8.868553e-02,-1.620060e-02,-1.975502e+01
75%,4.583195e+01,4.756470e+00,2.074482e+01,2.568239e+01,1.807023e+01,1.406954e+01,4.861539e+00,3.465567e+00,6.373827e+00,2.293000e+00,...,8.117053e-01,1.335385e+00,5.535190e-01,-1.597634e-01,9.770071e+00,1.036067e+01,1.002263e+01,1.561588e-01,6.280017e-02,-1.111857e+01
max,2.682446e+02,4.237981e+01,1.540855e+02,1.661700e+02,1.291188e+02,8.975859e+01,3.003588e+01,1.766028e+01,4.049660e+01,2.529795e+01,...,4.469280e+00,9.516260e+00,3.311444e+00,5.954419e+01,3.766922e+01,4.983179e+01,4.439345e+01,1.528149e+00,3.399830e-01,1.515419e+03


In [15]:
tp_.describe()

,总资产(亿元),货币资产(亿元),净资产(亿元),总债务(亿元),带息债务(亿元),净债务(亿元),经营活动现金流(亿元),投资活动现金流(亿元),筹资活动现金流(亿元),主营业务收入(亿元),...,净资产回报率(%),流动比率,速动比率,存货周转率,资产负债率,短期债务/总债务,带息债务/总投入资本,货币资金/短期债务,货币资金/总债务,获息倍数
count,5.100000e+02,5.100000e+02,5.100000e+02,5.100000e+02,5.100000e+02,5.100000e+02,5.100000e+02,5.100000e+02,5.100000e+02,5.100000e+02,...,5.100000e+02,5.100000e+02,5.100000e+02,5.100000e+02,5.100000e+02,5.100000e+02,5.100000e+02,5.100000e+02,5.100000e+02,5.100000e+02
mean,-9.819422e-14,-1.731077e-15,5.494864e-14,7.267041e-14,1.681618e-14,1.258079e-14,-3.291485e-16,-1.100645e-15,-7.290900e-15,-8.637971e-15,...,3.263620e-15,5.694791e-16,4.419123e-16,1.242057e-14,3.393886e-14,-3.280339e-14,-1.487263e-14,4.837089e-16,-1.195939e-16,2.250052e-15
std,1.115325e+02,1.317085e+01,4.884508e+01,6.867858e+01,4.907353e+01,4.208347e+01,8.223404e+00,1.044166e+01,1.225584e+01,3.294822e+01,...,4.090749e+00,1.025959e+00,5.784873e-01,9.069759e+01,1.305759e+01,1.893972e+01,1.568246e+01,3.016366e-01,1.159368e-01,1.362420e+01
min,-1.547063e+02,-1.403428e+01,-6.357559e+01,-8.689370e+01,-5.660534e+01,-6.739601e+01,-3.164306e+01,-3.570763e+01,-3.819683e+01,-2.922450e+01,...,-1.297580e+01,-1.431279e+00,-9.882863e-01,-4.180750e+01,-3.752779e+01,-4.037736e+01,-4.192271e+01,-4.273803e-01,-1.677664e-01,-2.061794e+01
25%,-8.968791e+01,-9.431194e+00,-3.475279e+01,-5.389504e+01,-3.713337e+01,-3.175402e+01,-4.007343e+00,-4.449518e+00,-7.423486e+00,-2.102806e+01,...,-2.800100e+00,-6.761540e-01,-4.195113e-01,-4.088455e+01,-8.227088e+00,-1.432726e+01,-1.123623e+01,-2.125995e-01,-8.605363e-02,-4.669811e+00
50%,-3.031366e+01,-4.348586e+00,-1.271150e+01,-2.249677e+01,-1.630584e+01,-1.285685e+01,-9.598116e-01,3.210736e+00,-2.419200e+00,-1.188348e+01,...,-7.429002e-01,-2.935790e-01,-1.268363e-01,-3.622865e+01,-3.818384e-01,-1.589813e+00,2.749431e-01,-6.234477e-02,-2.546908e-02,-3.601486e+00
75%,6.486096e+01,5.484409e+00,1.903390e+01,3.788810e+01,2.029042e+01,1.794716e+01,3.983999e+00,7.633885e+00,5.742544e+00,7.082763e+00,...,2.050575e+00,4.784710e-01,2.904637e-01,-8.648003e+00,9.057012e+00,1.255774e+01,1.134527e+01,1.375650e-01,5.260398e-02,-1.094011e+00
max,4.289894e+02,6.098161e+01,2.131756e+02,2.493261e+02,1.817618e+02,1.435618e+02,2.703601e+01,2.008071e+01,5.072891e+01,1.487937e+02,...,1.359200e+01,3.979221e+00,1.895714e+00,6.854565e+02,3.448761e+01,5.356984e+01,3.815599e+01,1.314159e+00,3.680209e-01,1.022267e+02


In [16]:
indestry_.describe()

,总资产(亿元),货币资产(亿元),净资产(亿元),总债务(亿元),带息债务(亿元),净债务(亿元),经营活动现金流(亿元),投资活动现金流(亿元),筹资活动现金流(亿元),主营业务收入(亿元),...,净资产回报率(%),流动比率,速动比率,存货周转率,资产负债率,短期债务/总债务,带息债务/总投入资本,货币资金/短期债务,货币资金/总债务,获息倍数
count,5.188000e+03,5.188000e+03,5.188000e+03,5.188000e+03,5.188000e+03,5.188000e+03,5.188000e+03,5.188000e+03,5.188000e+03,5.188000e+03,...,5.188000e+03,5.188000e+03,5.188000e+03,5.188000e+03,5.188000e+03,5.188000e+03,5.188000e+03,5.188000e+03,5.188000e+03,5.188000e+03
mean,-6.408170e-14,-2.439923e-14,1.681581e-14,-9.932670e-14,-5.782404e-15,-2.686860e-14,-7.702911e-15,6.547320e-15,3.382200e-15,8.683194e-15,...,-2.326709e-14,1.277570e-16,-9.266339e-16,-1.483813e-14,-8.685111e-14,-4.035220e-14,-4.371043e-14,1.201226e-16,1.507778e-16,4.899704e-16
std,7.716758e+01,1.211971e+01,2.766899e+01,5.415128e+01,3.344742e+01,2.689224e+01,6.909852e+00,6.492092e+00,8.884632e+00,6.166075e+01,...,6.922160e+00,6.260300e-01,4.420487e-01,1.649681e+01,1.430365e+01,1.830764e+01,1.830509e+01,2.225223e-01,1.308980e-01,1.364325e+01
min,-1.103249e+02,-1.331030e+01,-3.760291e+01,-6.745643e+01,-3.870409e+01,-7.562728e+01,-2.819134e+01,-2.385921e+01,-3.350267e+01,-5.627463e+01,...,-2.687534e+01,-1.342241e+00,-9.076650e-01,-7.187776e+00,-4.342687e+01,-5.586433e+01,-4.785039e+01,-3.717450e-01,-2.184217e-01,-1.142862e+02
25%,-5.947485e+01,-8.821081e+00,-1.920009e+01,-4.218649e+01,-2.587048e+01,-1.982070e+01,-3.162305e+00,-2.673735e+00,-4.586026e+00,-4.159026e+01,...,-4.371620e+00,-4.250414e-01,-3.196150e-01,-5.824651e+00,-9.918545e+00,-1.259578e+01,-1.366682e+01,-1.600099e-01,-9.407446e-02,-5.021624e+00
50%,-1.725084e+01,-3.665027e+00,-6.666051e+00,-1.420362e+01,-9.478631e+00,-7.939092e+00,-1.091418e+00,1.883429e+00,-1.509505e+00,-2.242134e+01,...,-9.893449e-01,-1.074414e-01,-6.716502e-02,-3.763876e+00,1.117380e+00,2.173970e+00,1.711555e-01,-4.635296e-02,-2.854027e-02,-3.626924e+00
75%,4.323477e+01,5.341243e+00,1.234252e+01,2.877907e+01,1.639829e+01,1.190706e+01,2.915339e+00,4.273567e+00,4.005890e+00,1.779998e+01,...,3.950730e+00,3.075586e-01,2.423850e-01,-3.692762e-01,1.060048e+01,1.465869e+01,1.367398e+01,1.097266e-01,6.459130e-02,-3.135494e-01
max,3.171852e+02,5.637031e+01,1.405833e+02,1.957358e+02,1.501767e+02,9.879271e+01,2.902928e+01,2.055484e+01,3.666524e+01,3.147427e+02,...,2.699266e+01,2.374359e+00,1.508835e+00,2.754582e+02,3.906043e+01,2.852927e+01,4.914971e+01,9.074983e-01,4.622475e-01,1.632300e+02


In [17]:
pca = PCA(n_components=3,)
ci__ = pca.fit_transform(ci_.iloc[:,2:],)
pca = PCA(n_components=3,)
tp__ = pca.fit_transform(tp_.iloc[:,2:])
pca = PCA(n_components=3,)
indestry__ = pca.fit_transform(indestry_.iloc[:,2:])
pca = PCA(n_components=3,)
fi__ = pca.fit_transform(indestry_.iloc[:,2:])

In [18]:
pca = PCA(n_components=3,)
ci__ = pca.fit_transform(ci_.iloc[:,2:],)
pca = PCA(n_components=3,)
tp__ = pca.fit_transform(tp_.iloc[:,2:])
pca = PCA(n_components=3,)
indestry__ = pca.fit_transform(indestry_.iloc[:,2:])
pca = PCA(n_components=3,)
fi__ = pca.fit_transform(indestry_.iloc[:,2:])

In [19]:
ina = np.random.permutation(ci__.shape[0])[:100]
inb = np.random.permutation(tp__.shape[0])[:100]
inc = np.random.permutation(indestry__.shape[0])[:100]
ind = np.random.permutation(fi__.shape[0])[:100]

In [20]:
ci___ = ci__[ina]
tp___ = tp__[inb]
indestry___ = indestry__[inc]
fi___ = fi__[ind]

In [21]:
ax = plt.subplot(111, projection='3d')
# ax.scatter(ci___[:,0], ci___[:,1], ci___[:,2], c='g')
# ax.scatter(tp___[:,0], tp___[:,1], tp___[:,2], c='r')
ax.scatter(indestry___[:,0], indestry___[:,1], indestry___[:,2], c='b')
ax.scatter(fi___[:,0], fi___[:,1], fi___[:,2], c='orange')

In [284]:
ci_.iloc[ina,:].to_excel(r'D:\pycharm_folder\xijing\company\2018-06-07\data\Debt issuing company 2018 report\result\a\城投.xlsx',index=False)
ci_.iloc[ina,:].describe().to_excel(r'D:\pycharm_folder\xijing\company\2018-06-07\data\Debt issuing company 2018 report\result\a\城投_.xlsx',)
tp_.iloc[inb,:].to_excel(r'D:\pycharm_folder\xijing\company\2018-06-07\data\Debt issuing company 2018 report\result\a\交通.xlsx',index=False)
tp_.iloc[inb,:].describe().to_excel(r'D:\pycharm_folder\xijing\company\2018-06-07\data\Debt issuing company 2018 report\result\a\交通_.xlsx',)
indestry_.iloc[inc,:].to_excel(r'D:\pycharm_folder\xijing\company\2018-06-07\data\Debt issuing company 2018 report\result\a\基础设施.xlsx',index=False)
indestry_.iloc[inc,:].describe().to_excel(r'D:\pycharm_folder\xijing\company\2018-06-07\data\Debt issuing company 2018 report\result\a\基础设施_.xlsx',)

In [167]:
ax = plt.subplot(111, projection='3d')
ax.scatter(ci___[:,0], ci___[:,1], ci___[:,2], c='g')
ax.scatter(tp___[:,0], tp___[:,1], tp___[:,2], c='r')
ax.scatter(indestry___[:,0], indestry___[:,1], indestry___[:,2], c='b')
# ax.scatter(fi___[:,0], fi___[:,1], fi___[:,2], c='orange')

In [67]:
aa = pd.DataFrame(pca.components_,columns=ci_.columns[2:])

In [68]:
aa

,总资产(亿元),货币资产(亿元),净资产(亿元),总债务(亿元),带息债务(亿元),净债务(亿元),经营活动现金流(亿元),投资活动现金流(亿元),筹资活动现金流(亿元),主营业务收入(亿元),...,净资产回报率(%),流动比率,速动比率,存货周转率,资产负债率,短期债务/总债务,带息债务/总投入资本,货币资金/短期债务,货币资金/总债务,获息倍数
0,0.676650,0.085482,0.148017,0.484744,0.277161,0.191679,0.017298,-0.023662,0.014575,0.390240,...,-0.001825,-0.000746,-0.000732,-0.004028,0.059481,-0.026317,0.079289,-0.000130,-0.000153,-0.010640
1,-0.271254,0.008509,-0.100696,-0.166625,-0.136052,-0.144561,0.016172,0.009170,-0.026028,0.899210,...,0.033214,-0.001684,0.000038,0.030206,-0.001261,0.180446,-0.018148,0.000042,0.000489,0.011749
2,0.036956,0.012372,0.053219,-0.000342,-0.008823,-0.021195,-0.009151,-0.032935,0.061781,-0.038465,...,0.049875,0.001529,0.001077,0.006994,-0.019850,-0.018814,-0.033661,0.000402,0.000232,0.046716


In [71]:
from sklearn.externals import joblib

In [89]:
# aaa = pd.DataFrame(ci___.T)
# aaa.to_excel(r'D:\pycharm_folder\xijing\company\2018-06-07\data\Debt issuing company 2018 report\result\ci.xlsx')
# bbb = pd.DataFrame(tp___.T)
# bbb.to_excel(r'D:\pycharm_folder\xijing\company\2018-06-07\data\Debt issuing company 2018 report\result\tp.xlsx')
# ccc = pd.DataFrame(indestry___.T)
# ccc.to_excel(r'D:\pycharm_folder\xijing\company\2018-06-07\data\Debt issuing company 2018 report\result\indesrty.xlsx')

In [114]:
pca = PCA(n_components=2,)
ci__ = pca.fit_transform(ci_.iloc[:,2:],)
pca = PCA(n_components=2,)
tp__ = pca.fit_transform(tp_.iloc[:,2:])
pca = PCA(n_components=2,)
indestry__ = pca.fit_transform(indestry_.iloc[:,2:])
pca = PCA(n_components=2,)
fi__ = pca.fit_transform(indestry_.iloc[:,2:])

In [115]:
ci___ = ci__[np.random.permutation(ci__.shape[0])[:500]]
tp___ = tp__[np.random.permutation(tp__.shape[0])[:500]]
indestry___ = indestry__[np.random.permutation(indestry__.shape[0])[:500]]
fi___ = fi__[np.random.permutation(tp__.shape[0])[:100]]

In [117]:
plt.scatter(ci___[:,0],ci___[:,1],c='g')
plt.scatter(tp___[:,0],tp___[:,1],c='r')
plt.scatter(indestry___[:,0],indestry___[:,1],c='b')